In [ ]:
from typing import List, Optional, Union, Dict
from enum import Enum
from pydantic import BaseModel, Field, FilePath, TypeAdapter

class NoExtraBaseModel(BaseModel, extra='forbid'):
    pass

In [ ]:
# Cubic models
class vdW1(NoExtraBaseModel):
    a: float = Field(description="The parameter a in vdW")
    b: float = Field(description="The parameter b in vdW")
        
class vdW(NoExtraBaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', description='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', description='The critical pressures in Pa')
        
# Helper types for cubic models
class AlphaOptions(str, Enum):
    Twu = "Twu"
    MathiasCopeman = "Mathias-Copeman"
    
class CubicAlpha(NoExtraBaseModel):
    type_: AlphaOptions = Field(alias='type', description='The type of the alpha function')
    c: List[float] = Field(min_length=3, max_length=3, description='The set of coefficients')
        
class PR(NoExtraBaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', description='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', description='The critical pressures in Pa')
    acentric: List[float] = Field(description='The acentric factors')
    kmat: Optional[List[List[float]]] = Field(default=None, description="The NxN matrix of kij factors")
        
SRK = PR

class GenericCubicEnum(str, Enum):
    PR = "PR"
    SRK = "SRK"
    
class GenericCubic(NoExtraBaseModel):
    type_: GenericCubicEnum = Field(alias='type')
    TcK: List[float] = Field(alias='Tcrit / K', description='The critical temperatures in K')
    pcK: List[float] = Field(alias='pcrit / Pa', description='The critical pressures in Pa')
    acentric: List[float] = Field(description='The acentric factors')
    alpha: List[CubicAlpha] = Field(default=None, description='The alpha function used in the attractive part')
    kmat: Optional[List[List[float]]] = Field(default=None, description="The NxN matrix of kij factors")

In [ ]:
class CPAPure(NoExtraBaseModel):
    a0i: float = Field(alias="a0i / Pa m^6/mol^2")
    bi: float = Field(alias="bi / m^3/mol")
    c1: float = Field(alias="c1")
    Tc: float = Field(alias="Tc / K")
    epsAB: float = Field(alias="epsABi / J/mol")
    betaAB: float = Field(alias="betaABi")
    class_: str = Field(alias="class")
    
class CPACubicEnum(str, Enum):
    PR = "PR"
    SRK = "SRK"
    
class CPARadialDistEnum(str, Enum):
    CS = "CS"
    KG = "KG"
    
class CPA(NoExtraBaseModel):
    cubic: CPACubicEnum
    pures: List[CPAPure]
    radial_dist: CPARadialDistEnum
    R: float = Field(alias="R_gas / J/mol/K")
        

In [ ]:
class multifluid(NoExtraBaseModel):
    components: List[FilePath | str]
    root: Optional[str]
    departure: FilePath | List[dict]
    BIP: FilePath | List[dict]
    flags: Optional[object]

In [ ]:
# Some model potentials requiring arguments
class SW_EspindolaHeredia2009(NoExtraBaseModel):
    lambda_: float = Field(alias='lambda', description='The well width, where the attractive part goes includes r=(sigma,lambda*sigma)')
    
class EXP6_Kataoka1992(NoExtraBaseModel):
    alpha: float = Field(description='The alpha parameter controlling the hardness of the EXP-6 potential')
        
class Mie_Pohl2023(NoExtraBaseModel):
    lambda_r: float = Field(description='The repulsive exponent; the attractive exponent is 6')
        
class TwoCLJFAuthors(str, Enum):
    Mecke = "2CLJF_Mecke"
    Lisal = "2CLJF_Lisal"
    
class TwoCJLF_Dipole(NoExtraBaseModel):
    author: TwoCLJFAuthors
    Lstar: float = Field(alias='L^*', description="The dimensionless separation of centers")
    mustar2: float = Field(alias='(mu^*)^2', description='The dipole moment squared, expressed in simulation units')
        
class TwoCJLF_Quadrupole(NoExtraBaseModel):
    author: TwoCLJFAuthors
    Lstar: float = Field(alias='L^*', description="The dimensionless separation of centers")
    Qstar2: float = Field(alias='(Q^*)^2', description='The quadrupole moment squared, expressed in simulation units')

In [ ]:
from typing import Literal

class IdealHelmholtzConstant(NoExtraBaseModel):
    type_: Literal["Constant"] = Field(alias="type", )
    R: float
    a: float 
        
class IdealHelmholtzLead(NoExtraBaseModel):
    # ln(rho) + a_1 + a_2/T
    type_: Literal["Lead"] = Field(alias="type", )
    R: float
    a_1: float 
    a_2: float 

class IdealHelmholtzLogT(NoExtraBaseModel):
    # a*ln(T)
    type_: Literal["LogT"] = Field(alias="type", )
    R: float
    a: float 
        
class IdealHelmholtzPowerT(NoExtraBaseModel):
    # sum_i n_i * T^i
    type_: Literal["PowerT"] = Field(alias="type", )
    R: float
    n: List[float]
    t: List[float]
        
class IdealHelmholtzPlanckEinstein(NoExtraBaseModel):
    type_: Literal["PlanckEinstein"] = Field(alias="type", )
    R: float
    n: List[float]
    theta: List[float]
                
class IdealHelmholtzPlanckEinsteinGeneralized(NoExtraBaseModel):
    type_: Literal["PlanckEinsteinGeneralized"] = Field(alias="type", )
    R: float
    n: List[float]
    c: List[float]
    d: List[float]
    theta: List[float]

class IdealHelmholtzGERG2004Cosh(NoExtraBaseModel):
    type_: Literal["GERG2004Cosh"] = Field(alias="type", )    
    R: float
    n: List[float]
    theta: List[float]
        
class IdealHelmholtzGERG2004Sinh(NoExtraBaseModel):
    type_: Literal["GERG2004Sinh"] = Field(alias="type", )
    R: float
    n: List[float]
    theta: List[float]
        
class IdealHelmholtzCp0Constant(NoExtraBaseModel):
    type_: Literal["Cp0Constant"] = Field(alias="type", )
    R: float
    c: float|List[float]
    T_0: float|List[float]
        
class IdealHelmholtzCp0PowerT(NoExtraBaseModel):
    type_: Literal["Cp0PowerT"] = Field(alias="type", )
    R: float
    c: float|List[float]
    t: float|List[float]
    T_0: float|List[float]
        
class IdealHelmholtzPure(NoExtraBaseModel):
    R: float
    terms: List[Union[
        IdealHelmholtzConstant, IdealHelmholtzLead, IdealHelmholtzLogT,
        IdealHelmholtzPowerT,IdealHelmholtzPlanckEinstein,
        IdealHelmholtzPlanckEinsteinGeneralized,
        IdealHelmholtzGERG2004Cosh, IdealHelmholtzGERG2004Sinh,
        IdealHelmholtzCp0Constant, IdealHelmholtzCp0PowerT
    ]]
        
IdealHelmholtz = TypeAdapter(List[IdealHelmholtzPure])
# IdealHelmholtz.json_schema()

# import glob, teqp
# for path in glob.glob(teqp.get_datapath()+'/dev/fluids/*.json'):
#     jig = teqp.convert_CoolProp_idealgas(path, 0)
#     aig = teqp.IdealHelmholtz([jig])
#     IdealHelmholtz.validate_python([jig])

In [ ]:
class ECSHuberEly1994ReferenceFluid(NoExtraBaseModel):
    name: str = Field(description="The name of the fluid, could be an absolute path to the .json of a multifluid EOS")
    acentric: float = Field(description="The acentric factor")
    Z_crit: float  = Field(description="The critical compressibility factor; Z = p/(rho*R*T)")
    T_crit: float = Field(alias="T_crit / K", description="The critical temperature")
    rhomolar_crit: float = Field(alias="rhomolar_crit / mol/m^3", description="The critical density")
        
class ECSHuberEly1994Fluid(NoExtraBaseModel):
    name: Optional[str] = Field(default=None, description="The name of the fluid, could be an absolute path to the .json of a multifluid EOS")
    acentric: float = Field(description="The acentric factor")
    Z_crit: float  = Field(description="The critical compressibility factor; Z = p/(rho*R*T)")
    T_crit: float = Field(alias="T_crit / K", description="The critical temperature")
    rhomolar_crit: float = Field(alias="rhomolar_crit / mol/m^3", description="The critical density")
    f_T_coeffs: List[float] = Field(min_length=2, max_length=2, description='The array containing the coefficients alpha_1 and alpha_2 used in f');
    h_T_coeffs: List[float] = Field(min_length=2, max_length=2, description='The array containing the coefficients beta_1 and beta_2 used in h');
        
class MultifluidECSHuberEly1994(NoExtraBaseModel):
    reference_fluid: ECSHuberEly1994ReferenceFluid
    fluid: ECSHuberEly1994Fluid

In [ ]:
class PCSAFTCoeff(NoExtraBaseModel):
    name: str
    m: float
    sigma_Angstrom: float
    epsilon_over_k: float
    BibTeXKey: str
    mustar2: Optional[float] = Field(alias='(mu^*)^2', default=None, description="The reduced dipole moment squared, as defined by Gross and co-workers. Watch out for missing factor of Coulomb's constant")
    nmu: Optional[float] = Field(default=None, description="The number of dipole moments")
    Qstar2: Optional[float] = Field(default=None, alias='(Q^*)^2', description="The reduced quadrupolar moment squared, as defined by Gross and co-workers. Watch out for missing factor of Coulomb's constant")
    nQ: Optional[float] = Field(default=None, description="The number of quadrupolar moments")
    
class PCSAFTABEnum(str, Enum):
    Liang_IECR_2012 = "Liang-IECR-2012"
    Liang_IECR_2014 = "Liang-IECR-2014"
    
class BasePCSAFT(NoExtraBaseModel):
    kmat: Optional[List[List[float]]] = Field(default=None, description="The NxN matrix of kij factors")
    ab: Optional[PCSAFTABEnum] = Field(default=None, description="The alternative source for a and b universal parameters")
        
class PCSAFTWithNames(BasePCSAFT):
    names: List[str]

class PCSAFTWithCoeffs(BasePCSAFT):
    coeffs: Optional[List[PCSAFTCoeff]]
        
PCSAFT = TypeAdapter(Union[PCSAFTWithNames, PCSAFTWithCoeffs])

In [ ]:
class SAFTVRMieCoeffBase(NoExtraBaseModel):
    name: str
    m: float
    epsilon_over_k: float
    BibTeXKey: str
    lambda_r: float
    lambda_a: float
    
    # Polar things
    mu_Cm: Optional[float] = Field(default=None, description="The dipole moment, in Debye")
    mu_D: Optional[float] = Field(default=None, description="The dipole moment, in C*m")
    Q_Cm2: Optional[float] = Field(default=None, description="The quadrupolar moment, in Debye")
    Q_DA: Optional[float] = Field(default=None, description="The quadrupolar moment, in C*m")
    nmu: Optional[float] = Field(default=None, description="The number of dipole moments, consider it a weighting parameter")
    nQ: Optional[float] = Field(default=None, description="The number of quadrupolar moments, consider it a weighting parameter")
        
class SAFTVRMieCoeffsigmaAngstrom(SAFTVRMieCoeffBase):
    sigma_Angstrom: float

class SAFTVRMieCoeffsigmaMeter(SAFTVRMieCoeffBase):
    sigma_m: float
    
class BaseSAFTVRMie(NoExtraBaseModel):
    kmat: Optional[List[List[float]]] = Field(default=None, description="The NxN matrix of kij factors")
    polar_model: Optional[str] = None
    SAFTVRMie_flags: Optional[Dict] = None
    polar_flags: Optional[Dict] = None
        
class SAFTVRMieWithNames(BaseSAFTVRMie):
    names: List[str]

class SAFTVRMieWithCoeffs(BaseSAFTVRMie):
    coeffs: List[Union[SAFTVRMieCoeffsigmaAngstrom,SAFTVRMieCoeffsigmaMeter]]
        
SAFTVRMie = TypeAdapter(Union[SAFTVRMieWithNames, SAFTVRMieWithCoeffs])
# SAFTVRMieWithCoeffs.validate({"coeffs":[{"BibTeXKey":"me","epsilon_over_k":100,"lambda_a":6,"lambda_r": 12,"m":1.0,"name":"Stockmayer126","sigma_m":3e-10}]})
# SAFTVRMie.validate_python({"coeffs":[{"BibTeXKey":"Paricaud","Q_DA":1.4151,"epsilon_over_k":299.424, "lambda_a":6.0,"lambda_r":21.7779,"m":1.3656,"mu_D":2.2814,"nQ":1.0,"name":"R1234YF","nmu":1.0,"sigma_Angstrom":4.5307}],"polar_model":"GrayGubbins+GubbinsTwu"})

In [ ]:
import json
    
examples = [
    {"kind": "vdW1", "model":{"a": 1.0, "b": 2.0}},
    {"kind": "GenericCubic", "model": {"type": "PR", "Tcrit / K": [190], "pcrit / Pa": [3.5e6], "acentric": [0.11], "alpha": [{"type": "Twu", "c":[1,2,3]}]}},
]
for example in examples:
    klass = locals()[example['kind']]
    klass.model_validate(example['model'])
    
# Put each of the schemas into a dictionary where the key is the 
# model kind (as a string) with the value being the schema for this particular 
# model kind
schemas = {}
for klass in [vdW1, vdW, PR, SRK, (GenericCubic, "cubic"),
              CPA, 
              (PCSAFT, "PCSAFT"),
              # multifluid
              (MultifluidECSHuberEly1994, "multifluid-ECS-HuberEly1994"),
              SW_EspindolaHeredia2009, 
              EXP6_Kataoka1992, 
              Mie_Pohl2023, 
              (TwoCJLF_Dipole, "2CLJF-Dipole"),
              (TwoCJLF_Quadrupole, "2CLJF-Quadrupole"),
              (IdealHelmholtz, "IdealHelmholtz"),
              (SAFTVRMie, "SAFT-VR-Mie")
             ]:
    if isinstance(klass, tuple) and len(klass) == 2:
        klass_, alias = klass
        if isinstance(klass_, TypeAdapter):
            schemas[alias] = klass_.json_schema()
        else:
            schemas[alias] = klass_.model_json_schema()
    else:
        if isinstance(klass, TypeAdapter):
            schemas[klass.__name__] = klass.json_schema()
        else:
            schemas[klass.__name__] = klass.model_json_schema()
    
with open("model_schemas.json", 'w') as fp:
    fp.write(json.dumps(schemas, indent=2))

In [ ]:
import json
def string_to_hexchunk(jsonstring):
    
    def to_chunks(l, n):
        if n < 1:
            n = 1
        return [l[i:i + n] for i in range(0, len(l), n)]
    
    try:
        h = ["0x{:02x}".format(ord(b)) for b in jsonstring] + ['0x00']
    except TypeError:
        h = ["0x{:02x}".format(int(b)) for b in jsonstring] + ['0x00']

    # Break up the file into lines of 16 hex characters
    # because some compilers don't like VERY long lines
    chunks = to_chunks(h, 16)

    # Put the lines back together again
    # The chunks are joined together with commas, and then EOL are used to join the rest
    hex_string = '{' + ',\n'.join([', '.join(chunk) for chunk in chunks]) + '}'
    return hex_string
            
chunk = string_to_hexchunk(json.dumps(schemas, indent=0))
INFO = """// Due to limitations in MSVC, very long string literals are 
// not allowed. Thus the string must be re-encoded as binary. The 
// contents of the string are in the JSON file next to this file

#include "nlohmann/json.hpp"
#include <string>
"""
with open('model_schemas.cpp','w') as fp:
    fp.write(INFO)
    fp.write('extern const auto model_schema_library = nlohmann::json::parse(std::string(\n' + chunk + '\n)); ')

In [ ]:
import tarfile, os

path = '../dev/model_schemas.tar.xz'
with tarfile.open(path, mode='w:xz') as tar:
    tar.add('model_schemas.json')
    tar.add('model_schemas.cpp')
    
# from json_schema_for_humans.generate import generate_from_filename
# from json_schema_for_humans.generation_configuration import GenerationConfiguration

# config = GenerationConfiguration(copy_css=True, expand_buttons=True, template_name='js_offline', link_to_reused_ref=False)

# import json, io, os
# for k, schema in json.load(open('model_schemas.json')).items():
    
#     with open('tmp.tmp','w') as fp:
#         fp.write(json.dumps(schema))
#     os.makedirs(k, exist_ok=True)
#     generate_from_filename('tmp.tmp', f"{k}_schema_doc.html", config=config)
#     os.remove('tmp.tmp')
    
os.remove('model_schemas.json')
os.remove('model_schemas.cpp')